# Security and Encryption

## Known Issues

---

If the app is to be multitenant, the api keys of the portfolio must be known to the worker. These cannot be passed through redis unless there is some sort of encryption

Possible magic bullet: https://blog.heroku.com/securing-celery

But after pip installing, celery breaks unless the environment already has a KOMBU_FERNET_KEY variable

So trying this: https://github.com/roverdotcom/kombu-encrypted-serializer

---

In [2]:
import os
from cryptography.fernet import Fernet

key = Fernet.generate_key()
os.environ['KOMBU_FERNET_KEY'] = str(key)

In [ ]:
import os

from celery import Celery
from kombu_encrypted_serializer import setup_encrypted_serializer

TEST_KEY = 'WgFNqB8eokKER0aFxEmfnK7qyZmGhGmxxOqccW3oZoM='
os.environ['KOMBU_ENCRYPTED_SERIALIZER_KEY'] = TEST_KEY

serializer_name = setup_encrypted_serializer(serializer='pickle')

app = Celery('tasks', broker='redis://', backend='redis://')
app.conf.update(
    CELERY_TASK_SERIALIZER=serializer_name,
    CELERY_RESULT_SERIALIZER=serializer_name,
    CELERY_ACCEPT_CONTENT=[serializer_name],
)


@app.task
def add(x, y):
    return x + y


@app.task
def give_it_back(data):
    return data